## Import Libraries

In [283]:
import numpy as np
import pandas as pd
from typing import Tuple
import pandas_ta as ta

# backtesting library
import vectorbt as vbt

# machine learning
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

# get data from TradingView
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()

## Import Data

In [4]:
interval = Interval.in_1_hour

raw_df = tv.get_hist(symbol='BTCUSDT', exchange='BINANCE', interval=interval , n_bars=100000)
raw_df

,symbol,open,high,low,close,volume
datetime,,,,,,
2022-01-01 07:00:00,BINANCE:BTCUSDT,46216.93,46731.39,46208.37,46656.13,1503.33095
2022-01-01 08:00:00,BINANCE:BTCUSDT,46656.14,46949.99,46574.06,46778.14,943.81539
2022-01-01 09:00:00,BINANCE:BTCUSDT,46778.14,46928.94,46721.96,46811.77,485.16860
2022-01-01 10:00:00,BINANCE:BTCUSDT,46811.77,46916.63,46760.12,46813.20,562.88971
2022-01-01 11:00:00,BINANCE:BTCUSDT,46813.21,46887.33,46591.23,46711.05,861.88389
...,...,...,...,...,...,...
2023-05-07 12:00:00,BINANCE:BTCUSDT,28840.03,28911.05,28824.88,28890.00,1109.88261
2023-05-07 13:00:00,BINANCE:BTCUSDT,28890.00,28936.00,28851.78,28914.55,1097.17087
2023-05-07 14:00:00,BINANCE:BTCUSDT,28914.56,28952.56,28892.04,28910.41,691.54569


## Add technical indicators

In [149]:
strategy_name = "Demo Strategy"
strategy1 = ta.Strategy(
    name='Demo Strategy',
    description='',
    ta=[
        {
            'kind': 'adx', 'length': 14
            , 'col_names': ('ADX_14', 'DMP_14', 'DMN_14')
            
        },
        {
            'kind': 'bbands', 'length': 20, 'std': 2, 
            'col_names': ('BBL', 'BBM', 'BBU', 'BBB', 'BBP')
        },
        {
            'kind': 'ema', 'length': 50
        },
        {
            'kind': 'ema', 'length': 100
        },
        {
            'kind': 'ema', 'length': 200
        },
        {
            "kind": "donchian", "lower_length": 12, "upper_length": 24
            , "col_names": ("DCL", "DCM", "DCU")
        },
        {
            "kind": "macd", "fast": 12, "slow": 26
            , "col_names": ("MACD", "MACD_H", "MACD_S")
        }
    ]
)

strat1 = raw_df.copy()
strat1.ta.strategy(strategy1)

# strat1.head(5)

In [150]:
# label for machine learning 
strat1['ret'] = strat1['close'].pct_change().shift(-1)

strat1['y'] = np.where(np.isnan(strat1['ret']), strat1['ret'], 
                       np.where(strat1['ret'] > 0
                                , 1
                                , 0)
                       )
# strat1

## Data Cleaning

In [141]:
def num_split_data(data: pd.DataFrame, train_size: float, test_size: float, out_of_sample_size: float, n_splits: int) -> Tuple[int, int, int]:
    """
    Calculate the size of training, testing, and out-of-sample data for time series cross-validation.

    Parameters
    ----------
    data : pd.DataFrame
        A pandas DataFrame containing the time series data.
    train_size : float
        The proportion of the data to use for training.
    test_size : float
        The proportion of the data to use for testing.
    out_of_sample_size : float
        The proportion of the data to use for out-of-sample prediction.
    n_splits : int
        The number of folds to use for cross-validation.

    Returns
    -------
    Tuple[int, int, int]
        A tuple containing the sizes of the training, testing, and out-of-sample data.
    """
    # Calculate the size of out-of-sample data
    rows_out_of_sample = (len(data)) * out_of_sample_size

    # Calculate the size of train/test dat
    train_test_size = ((len(data)) - rows_out_of_sample)
    num_of_days = train_test_size/24

    # Calculate the size of training and testing sets for each fold
    split_train_test_size = (train_test_size//n_splits)
    rows_train_size = round((split_train_test_size * train_size))
    rows_test_size = round((split_train_test_size * test_size))
    rows_out_of_sample = round(rows_out_of_sample)
    
    # Print the number of days, training size, and testing size for each fold
    print(f"Number of Days: {num_of_days}")
    print(f"Training Size for Each Fold: {rows_train_size}")
    print(f"Testing Size for Each Fold: {rows_test_size}")
    print(f"Out of Sample Size for Each Fold: {rows_out_of_sample}")
    
    return rows_train_size, rows_test_size, rows_out_of_sample

In [250]:
n_splits = 10

rows_train_size, rows_test_size, rows_out_of_sample = num_split_data(data=raw_df, train_size=0.80, test_size=0.20, 
                                                                     out_of_sample_size=0.1, n_splits=n_splits)

Number of Days: 442.2
Training Size for Each Fold: 849
Testing Size for Each Fold: 212
Out of Sample Size for Each Fold: 1179


In [393]:

X = strat1.drop(columns=['symbol', 'ret', 'y']).copy()
X.dropna(inplace=True)
y = strat1['y']
y = y[X.index] # matching index

# The training and validation datasets are utilized to explore and determine the optimal methodology
# The testing dataset, which represents unseen data, is then used once the best result has been achieved on the training and validation datasets.
X_train_val, X_test = X.iloc[:-rows_out_of_sample], X.iloc[-rows_out_of_sample:]
y_train_val, y_test = y.iloc[:-rows_out_of_sample], y.iloc[-rows_out_of_sample:]

## Split Train/Validation Dataset

In [271]:
# training for 1 month
# week * hrs
train_size = 28*24

# test for 1 week
test_size = 7*24

In [272]:
rf = RandomForestClassifier(random_state=42)

tscv = TimeSeriesSplit(n_splits=10, max_train_size=train_size, test_size=test_size)

# Perform cross-validation on the data
for k_fold, (X_train_val_index, y_train_val_index) in enumerate(tscv.split(X_train_val)) :
    # X_train_val = raw_indicators.iloc[X_train_val_index]
    # y_train_val = raw_indicators.iloc[y_train_val_index]
    train_data = X_train_val.iloc[X_train_val_index]
    test_data = X_train_val.iloc[y_train_val_index]
    print('--------------------')
    # number of fold
    print('Fold: ', k_fold+1)
    # Print the shapes of the training and testing sets for each fold
    print('Training data shape:', train_data.shape)
    print('Testing data shape:', test_data.shape)

    # Print the period of the training and testing sets for each fold 
    print('Training date:', train_data.index[0], "-", train_data.index[-1])
    print('Testing date:', test_data.index[0], "-", test_data.index[-1])

--------------------
Fold:  1
Training data shape: (672, 22)
Testing data shape: (168, 22)
Training date: 2022-12-11 12:00:00 - 2023-01-08 11:00:00
Testing date: 2023-01-08 12:00:00 - 2023-01-15 11:00:00
--------------------
Fold:  2
Training data shape: (672, 22)
Testing data shape: (168, 22)
Training date: 2022-12-18 12:00:00 - 2023-01-15 11:00:00
Testing date: 2023-01-15 12:00:00 - 2023-01-22 11:00:00
--------------------
Fold:  3
Training data shape: (672, 22)
Testing data shape: (168, 22)
Training date: 2022-12-25 12:00:00 - 2023-01-22 11:00:00
Testing date: 2023-01-22 12:00:00 - 2023-01-29 11:00:00
--------------------
Fold:  4
Training data shape: (672, 22)
Testing data shape: (168, 22)
Training date: 2023-01-01 12:00:00 - 2023-01-29 11:00:00
Testing date: 2023-01-29 12:00:00 - 2023-02-05 11:00:00
--------------------
Fold:  5
Training data shape: (672, 22)
Testing data shape: (168, 22)
Training date: 2023-01-08 12:00:00 - 2023-02-05 11:00:00
Testing date: 2023-02-05 12:00:00 - 

## Normalization

In [273]:
normalizers = [
    ('scaler', None),
    ('scaler', RobustScaler()),
    ('scaler', MinMaxScaler()),
    ('scaler', StandardScaler())
]

for i, normalizer in enumerate(normalizers):
    pipeline = Pipeline([
        normalizer,
        ('rf', RandomForestClassifier(random_state=42))
    ])

    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=train_size, test_size=test_size)

    score = cross_val_score(pipeline, X_train_val, y_train_val, scoring='roc_auc', cv=tscv)
    print("---------------------------")
    print(f'Normalizer: {type(normalizers[i][1]).__name__}')
    print('{}: {:.4f} +- {:.4f}'.format(type(pipeline).__name__, score.mean(), score.std()))
    print(score)

---------------------------
Normalizer: NoneType
Pipeline: 0.5215 +- 0.0379
[0.45240525 0.55080882 0.5112814  0.55159461 0.59019886 0.49537037
 0.50368533 0.4885848  0.55220798 0.5186013 ]
---------------------------
Normalizer: RobustScaler
Pipeline: 0.5216 +- 0.0400
[0.45014577 0.55816176 0.50461189 0.55159461 0.59417614 0.49337607
 0.50368533 0.48979013 0.55220798 0.5186013 ]
---------------------------
Normalizer: MinMaxScaler
Pipeline: 0.5223 +- 0.0387
[0.45145773 0.55720588 0.51340996 0.55159461 0.59019886 0.49337607
 0.50666194 0.48780488 0.55220798 0.5186013 ]
---------------------------
Normalizer: StandardScaler
Pipeline: 0.5214 +- 0.0392
[0.45014577 0.55816176 0.50659855 0.55159461 0.59019886 0.49337607
 0.50368533 0.48979013 0.55220798 0.5186013 ]


Select the Best Normalizer

In [280]:
# Define a parameter grid for the Random Forest model
param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [5, 10, None],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('rf', RandomForestClassifier(random_state=42))
])

tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=train_size, test_size=test_size)

grid_search = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=tscv)
grid_search.fit(X_train_val, y_train_val)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'rf__max_depth': 5, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 100}
Best score:  0.5475211143182627


In [394]:
# auc score
grid_search.score(X_train_val, y_train_val)

0.627826999324185

## Out of Sample

In [418]:
signal_df = X_test.copy()
signal_df['signal'] = pd.Series(data=grid_search.predict(X_test), index=X_test.index)

# convert signal as boolean
signal_df['signal'] = signal_df['signal'].apply(lambda x: True if x==1 else False)

# more realistic
signal_df['signal'] = signal_df['signal'].shift()
signal_df.dropna(inplace=True)

# create signal backtesting
signal_vectorbt = signal_df.ta.tsignals(signal_df.signal, asbool=True, append=True)

port = vbt.Portfolio.from_signals(signal_df.close,
                                    entries=signal_vectorbt.TS_Entries,
                                    exits=signal_vectorbt.TS_Exits,
                                    freq="1h",
                                    init_cash = 1000,
                                    size=0.1,
                                    # fees = 0.00075,
                                    # slippage = 0.001
                                )

port.plot().show()
port.stats()

Start                         2023-03-19 13:00:00
End                           2023-05-07 16:00:00
Period                           49 days 02:00:00
Start Value                                1000.0
End Value                             1046.113795
Total Return [%]                          4.61138
Benchmark Return [%]                     6.934594
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        12.649521
Max Drawdown Duration            22 days 10:00:00
Total Trades                                   75
Total Closed Trades                            75
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            53.333333
Best Trade [%]                           3.676184
Worst Trade [%]                         -4.181428
Avg Winning Trade [%]                    0.826647
Avg Losing Trade [%]                    -0.799559
